# Initialisation

In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
import tifffile
%matplotlib tk

# Importer fichier

In [3]:
path_to_tiff = os.path.join("..", "acquisition", "video_output_carac_150ms_1im_1um.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [4]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


71

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [5]:
frame_index = 0
first_frame = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    first_frame += 1
    print(frame_index)




1


# Traitement d'image

In [6]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)
print(img)

[[ 9 10 10 ... 11 11 11]
 [10 10 10 ... 11 11 11]
 [ 9  9 10 ... 11 12 12]
 ...
 [ 9  9 10 ...  8  8  8]
 [ 8  9  9 ...  8  8  8]
 [ 8  9  9 ...  8  9  8]]


# Sélection du point à tracker

In [7]:
def crop(img, x, y, crop_size=100):
    x_start = int(x - crop_size // 2)
    x_end = int(x + crop_size // 2)
    y_start = int(y - crop_size // 2)
    y_end = int(y + crop_size // 2)

    return img[y_start:y_end, x_start:x_end]

In [8]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze = 100

Please click on the point you want to select.
Selected point: (529.5649350649352, 491.5779220779219)


Debogueur

In [9]:
def visionneur(frame):
    plt.figure(figsize=(10, 5))
    plt.clf() 
    plt.imshow(frame, origin='lower', cmap='gray')
    plt.title('Grille Zoomée avec Position')
    plt.colorbar()  
    plt.show()

# Fit gaussien sur le point sélectionné

In [10]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [11]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [12]:
def localisateur_gaussien(intensity_grid, maxi):
    x = np.arange(intensity_grid.shape[0])
    y = np.arange(intensity_grid.shape[1])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    initial_x0 = x[max_idx[0]]
    initial_y0 = y[max_idx[1]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=1,
        sigma_y=1,
        offset=2
    )

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))
    
    x_position = result.params['x0'].value + maxi[0] - crop_sze/2
    y_position = result.params['y0'].value + maxi[1] - crop_sze/2

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [13]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Faire le crop et fit

In [14]:
def particle_tracker(image, x, y):
    image = denoise(image)

    cropped_img = crop(image, x, y, crop_sze)
    
    #Gérer plus qu'une particule
    cropped_img = np.array(cropped_img)
    max_index = np.argmax(cropped_img)
    max_coords = np.unravel_index(max_index, cropped_img.shape)

    nouveau_x = x - crop_sze // 2 + max_coords[1]
    nouveau_y = y - crop_sze // 2 + max_coords[0]
    
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze)    # Re-crop autour d'une seule particule

    result_fit = localisateur_gaussien(second_crop, [x, y])

    x_new, y_new = result_fit[0][0], result_fit[0][1]

    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [ ]:
position_list=[]
sigma_list = []
crop_frames = []
big_frames = []


for i in range(first_frame,actual_frames):
    tiff.seek(i)
    if np.sum(np.array(tiff))==0:
        position_list.append(np.array([np.nan,np.nan]))
        sigma_list.append(np.array([np.nan,np.nan]))
        print("hello")
    else:
        data = particle_tracker(img, x, y)
        position_list.append(data[2])
        x,y = position_list[i-first_frame]
        sigma_list.append(data[3])
        tiff.seek(i)
        img = np.array(tiff)
        big_frames.append(img)
        crop_frames.append(data[1])



hello
hello
hello


In [45]:
sigma_arr=np.array(sigma_list)
position_arr=np.array(position_list)

In [48]:
def calculate_msd_with_uncertainty(position, delta_x, delta_y):
    """
    Calcule les MSD avec propagation des incertitudes analytiques.
    """


    msd = []
    uncertainties = []
    for d in range(1, len(position_arr)):
        diff_pairs = np.zeros_like(position_arr[d:])
        dx = np.zeros_like(delta_x[d:])
        dy = np.zeros_like(delta_y[d:])

        # Masque pour détecter les NaN
        nan_mask_pos = np.isnan(position_arr[d:]) | np.isnan(position_arr[:-d])
        nan_mask_delta = np.isnan(delta_x[d:]) | np.isnan(delta_x[:-d])
        
        # Calculer les différences uniquement où nan_mask est False
        valid_mask_pos = ~nan_mask_pos.any(axis=1)  # Inverser le masque pour obtenir les position_arr valides
        valid_mask_delta = ~nan_mask_delta
        
        diff_pairs[valid_mask_pos] = position_arr[d:][valid_mask_pos, :] - position_arr[:-d][valid_mask_pos, :]

        distances_squared = np.sum(diff_pairs**2, axis=1)
        msd.append(np.mean(distances_squared))  
        
        dx[valid_mask_delta] = delta_x[d:][valid_mask_delta] + delta_x[:-d][valid_mask_delta]
        dy[valid_mask_delta] = delta_y[d:][valid_mask_delta] + delta_y[:-d][valid_mask_delta]
        term_x = 2 * (diff_pairs[:, 0]**2) * (dx**2)
        term_y = 2 * (diff_pairs[:, 1]**2) * (dy**2)
        
        # Propagation des incertitudes
        total_uncertainty = np.mean(term_x + term_y)
        uncertainties.append(np.sqrt(total_uncertainty))

    return np.array(msd), np.array(uncertainties)

calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

(array([  5.14796371,  11.14404418,  16.96930523,  22.74082078,
         24.53955178,  20.88561703,  22.42803964,  24.77903407,
         26.38223881,  32.77760687,  34.29617421,  35.1143051 ,
         36.52235097,  38.25030595,  39.33613997,  41.2066553 ,
         42.56366462,  43.39768311,  43.69251571,  43.38401289,
         42.57339661,  41.33829502,  41.03425015,  40.09270284,
         40.20249814,  39.72244326,  42.70522602,  43.06492953,
         46.71866301,  47.90600067,  49.12384176,  52.78447739,
         55.46194817,  58.59880119,  59.35030872,  61.89868958,
         64.99759135,  65.18113443,  69.56029217,  69.24881549,
         73.44960856,  83.48680562,  88.7742171 ,  97.64519306,
        102.00943288, 106.39746996, 108.52753152, 111.65391059,
        114.15646703, 115.90358503, 118.11814085, 122.21396919,
        122.58967173, 122.88600344, 124.69160074, 125.22851338,
        128.03919596, 129.13148754, 127.51727853, 118.93921568,
        140.46837217, 133.60947905, 127.

In [50]:
# Assuming msd_values is the result of your MSD calculation
msd_values, uncertainties = calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

# Time intervals for the x-axis (assuming a uniform time step)
time_intervals = np.arange(1, len(msd_values) + 1)

# Plotting the MSD
plt.figure(figsize=(8, 6))
plt.errorbar(time_intervals, msd_values, yerr=uncertainties, fmt='o', label='MSD', color='blue')

# Customize the plot
plt.title("MSD en fonction de l'intervalle de temps")
plt.xlabel("Intervalle de temps")
plt.ylabel("MSD")
plt.grid(True)
plt.legend()

# Show the plot
plt.show()

In [58]:
# Assuming msd_values is the result of your MSD calculation
msd_values, uncertainties = calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

cropped_msd = msd_values[:5]
cropped_inc = uncertainties[:5]

# Time intervals for the x-axis (assuming a uniform time step)
time_intervals = np.arange(1, len(cropped_msd) + 1)

# Perform a quadratic fit using np.polyfit
coeffs = np.polyfit(time_intervals, cropped_msd, 2)  # Degree 2 for quadratic
quadratic_fit = np.poly1d(coeffs)

# Generate fitted values for plotting the curve
fitted_msd = quadratic_fit(time_intervals)

# Plotting the MSD
plt.figure(figsize=(8, 6))
plt.errorbar(time_intervals, cropped_msd, yerr=cropped_inc, fmt='o', label='MSD', color='blue')
plt.plot(time_intervals, fitted_msd, label='Quadratic Fit', color='red', linestyle='--')


# Customize the plot
plt.title("MSD en fonction de l'intervalle de temps")
plt.xlabel("Intervalle de temps")
plt.ylabel("MSD")
plt.grid(True)
plt.legend()

# Show the plot
plt.show()

In [ ]:
print(f"Quadratic fit coefficients: {coeffs}")

r = (4 * 1.38 * 10**-23 * 300 / (6 * np.pi * 10**(-3) * coeffs[1]))  # Diffusion coefficient
print(r)

Quadratic fit coefficients: [-0.60346032  8.65875717 -3.22987091]
1.0146205374719122e-19


## anim crop

In [51]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()